In [2]:
import tensorflow as tf
import numpy as np
import cv2

In [9]:
# function to Load dataset

# Create a-z dataset
def load_az_dataset(path):
    # Intialise list data and lables
    data_az = []
    labels_az = []

    #Loop over the row of the a-z datset
    for row in open(path):
        # save each cell of the row in list
        row  = row.split(",")
        label = int(row[0])
        image = np.array([int(x) for x in row[1:]],dtype = "uint8")

        # convert image to 28*28 matrix
        image = np.reshape(image,(28,28))

        # add image and labels to dataset list
        data_az.append(image)
        labels_az.append(label)

    # Convert the list to numpy array
    data_az = np.array(data_az,dtype="float32")
    labels_az = np.array(labels_az,dtype="int")

    # return 2-tuple of the a-z data and labels
    return (data_az,labels_az)

# Load mnist dataset
def load_mnist_dataset():
    # Load mnist dataset and combine test and train data
    ((train,train_label),(test,test_label)) = tf.keras.datasets.mnist.load_data()
    data = np.vstack([train,test])
    labels = np.hstack([train_label,test_label])

    return (data,labels)

In [10]:
# Load az dataset
data_az , labels_az = load_az_dataset("/home/stark/Desktop/A_Z Handwritten Data.csv")


In [11]:
# Load 09 dataset
data_09 , labels_09 = load_mnist_dataset()

In [15]:
# Add 10 to labels_az so that first 10 labels can be 0-9 from mnist dataset
labels_az += 10

# Merge both dataset
data = np.vstack([data_09,data_az])
labels = np.hstack([labels_09,labels_az])

# Resize image to 32*32
data = [cv2.resize(image,(32,32)) for image in data]
data = np.array(data,dtype="float32")

# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]
data = np.expand_dims(data, axis=-1)
data /= 255.0